In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers





ModuleNotFoundError: No module named 'pinecone'

In [ ]:
PINECONE_API_KEY='pcsk_33dJwb_LuHpCRjZCZsktMRxbfJuVub362oFXVMLn8ddMGnXEpoXzL815Q9Vx72HBBAHr8M'
PINECONE_API_ENV="aws-starter"

# load data from pdf
def extract_data_pdf(path):
    loader=DirectoryLoader(path,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

extracted_data=load_pdf("data/")

# create text chunks
def text_split(pdf_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(pdf_data)
    return text_chunks

text_chunks=text_split(extracted_data)
print(len(text_chunks))

# download embedding model
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MinLM-L6-v2")
    return embeddings
embeddings=download_hugging_face_embeddings()
embeddings

query_result=embeddings.embed_query("Hello world")
print(len(query_result)) # 1-D matrix that represents the context (different from the GPT-2 architecture)

# init Pinecone
pinecone.init(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)
index_name="medical-chatbot" # with aws
# creating embeddings for each of the text chunks and storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name,embeddings)
query="What are Allergies"
docs=docsearch.similarity_search(query,k=3) # we recover 3 documents
print("Result",docs)

prompt_template="""
    Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    
    Context: {context}
    Question: {question}
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"]) # tells us which placeholders are for documents and query
chain_type_kwards={"prompt":PROMPT}
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",config={'max_new_tokens':512,'temperature':0.8})
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff", # multiple documents will be pasted together
                               retriever=docsearch.as_retriever(search_kwargs={'k':2}), # 2 most relevant documents will be returned
                               return_source_documents=True,chain_type_kwargs=chain_type_kwargs) # and they will go in the context placeholder
while True:
    user_input=input(f"Input prompt:")
    result=qa({"query":user_input}) # query replaces the place holder of question
    print("response:",result["result"])
    
    